<a href="https://colab.research.google.com/github/prakharraj11/UML501_MachineLearning/blob/main/ML_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Assignment 3

Q1: K-Fold Cross Validation for Multiple Linear Regression (Least Square Error Fit) Download the dataset regarding USA House Price Prediction from the
following link:
https://drive.google.com/file/d/1O_NwpJT-8xGfU_-3llUl2sgPu0xllOrX/view?usp=sharing
Load the dataset and Implement 5- fold cross validation for multiple linear regression (using least square error fit).

Steps:

a) Divide the dataset into input features (all columns except price) and output variable(price)

b) Scale the values of input features.

c) Divide input and output features into five folds.

d) Run five iterations, in each iteration consider one-fold as test set and remaining four sets as training set. Find the beta (𝛽) matrix, predicted values, and R2_score for each iteration using least square error fit.

e) Use the best value of (𝛽) matrix (for which R2_score is maximum), to train the regressor for 70% of data and test the performance for remaining 30% data.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('/content/USA_Housing.csv')

In [ ]:
df.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05


In [ ]:
df.count()

,0
Avg. Area Income,5000
Avg. Area House Age,5000
Avg. Area Number of Rooms,5000
Avg. Area Number of Bedrooms,5000
Area Population,5000
Price,5000


In [ ]:
X = df[['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms', 'Avg. Area Number of Bedrooms', 'Area Population']]
y = df[['Price']]

In [ ]:
X.head() #feature set

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population
0,79545.45857,5.682861,7.009188,4.09,23086.80050
1,79248.64245,6.002900,6.730821,3.09,40173.07217
2,61287.06718,5.865890,8.512727,5.13,36882.15940
3,63345.24005,7.188236,5.586729,3.26,34310.24283
4,59982.19723,5.040555,7.839388,4.23,26354.10947


In [ ]:
y.head() #traget variable

,Price
0,1.059034e+06
1,1.505891e+06
2,1.058988e+06
3,1.260617e+06
4,6.309435e+05


In [ ]:
# Scaling the value of input features
# scaled between -1 to 1
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
X

array([[ 1.02865969, -0.29692705,  0.02127433,  0.08806222, -1.31759867],
       [ 1.00080775,  0.02590164, -0.25550611, -0.72230146,  0.40399945],
       [-0.68462915, -0.11230283,  1.5162435 ,  0.93084045,  0.07240989],
       ...,
       [-0.48723454,  1.28447022, -2.17026949, -1.50025059, -0.29193658],
       [-0.05459152, -0.44669439,  0.14154061,  1.18205319,  0.65111608],
       [-0.28831272,  0.01521477, -0.19434166,  0.07185495,  1.04162464]])

Function to divide X into 5 folds

In [ ]:
def makeFolds(k):
  fold_size = X.shape[0] // k
  indices = np.arange(X.shape[0])
  np.random.shuffle(indices)
  indexwisefolds = []
  for i in range(k):
    test = indices[i*fold_size:(i+1)*fold_size] #a segment reserved for testing
    train = (np.concatenate([indices[(i+1)*fold_size:], indices[:i*fold_size]]))
    indexwisefolds.append((test,train))

  return indexwisefolds

In [ ]:
indexwisefolds = makeFolds(5)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,f1_score,mean_squared_error, recall_score,accuracy_score

In [ ]:
model = LinearRegression()
i = 1
bestr2 = -9999
best_coeff = []
for test_indices, train_indices in indexwisefolds:
  print(f'This is the iteration number {i}')
  X_train, y_train = X[train_indices], y.iloc[train_indices]
  X_test, y_test = X[test_indices], y.iloc[test_indices]
  model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  r2 = r2_score(y_test,y_pred)
  if r2>bestr2:
    bestr2 = r2
    best_coeff = model.coef_
  i+=1

print(f'The best r2 score is {bestr2}')
print(f'The best coefficients are {best_coeff}')

This is the iteration number 1
This is the iteration number 2
This is the iteration number 3
This is the iteration number 4
This is the iteration number 5
The best r2 score is 0.9237590595612615
The best coefficients are [[229133.75232573 164845.38755193 121059.34300962   1733.28060251
  150861.64793629]]


We can verify that the best r2 score is in the 3rd iteration

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

final_model = LinearRegression()
final_model.coef_ = np.array(best_coeff).reshape(-1)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)
r2 = r2_score(y_test,y_pred)
print(f'The final r2 score is {r2}')

The final r2 score is 0.9146818498916267


Concept of Validation set for Multiple Linear Regression (Gradient Descent
Optimization)

Consider the same dataset of Q1, rather than dividing the dataset into five folds, divide the dataset into training set (56%), validation set (14%), and test set (30%).

Consider four different values of learning rate i.e. {0.001,0.01,0.1,1}. Compute the values of regression coefficients for each value of learning rate after 1000 iterations.

For each set of regression coefficients, compute R2_score for validation and test set and find the best value of regression coefficients.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

In [5]:
newdf = pd.read_csv('/content/USA_Housing.csv')
X = newdf.drop('Price', axis=1)
y = newdf['Price']

In [6]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [7]:
# adding the bias column i.e y = B0*1 + Summation(Bi * xi)
X = np.c_[np.ones(X.shape[0]), X]

In [9]:
X_train,X_temp,y_train,y_temp = train_test_split(X,y,test_size = 0.44,random_state=42)
X_val,X_test,y_val,y_test = train_test_split(X_temp,y_temp,test_size=30/44, random_state=42)
#train/val/test split into 56/14/30

In [11]:
def gradient_descent(X,y,alpha,iter=1000):
  m,n = X.shape
  beta = np.zeros(n)
  for i in range(iter):
    y_pred = np.dot(X,beta) #getting the y values b0*1 + b1*x1 + b2*x2 + ....
    gradient = (1/m) * (X.T.dot(y_pred - y)) # finding gradient using formula 1/m * X^t(X*B - y)
    beta -= alpha * gradient
  return beta


In [12]:
# trying multiple learning rates
learning_rates = [0.001,0.01,0.1,1]
results = []
# finding the best r2 score using the validation data set
for lr in learning_rates:
  beta = gradient_descent(X_train,y_train,lr,iter = 1000) #finding beta values from the training set
  y_pred_val = X_val.dot(beta) #finding r2 score using the validation set
  r2_val = r2_score(y_val,y_pred_val)

  results.append((lr,beta,r2_val))

In [13]:
best_lr, best_beta, best_r2_val = max(results, key=lambda x: x[2]) #key 2 so that finds by r2 score of validation set
print(f"Best learning rate: {best_lr}")
print(f"Best beta values: {best_beta}")
print(f"Best R2 score on validation set: {best_r2_val}")

Best learning rate: 0.1
Best beta values: [1232180.27200919  230645.88389435  165328.94019375  120045.00851908
    2945.02108903  151375.22971285]
Best R2 score on validation set: 0.9199649194854793


Pre-processing and Multiple Linear Regression
Download the dataset regarding Car Price Prediction from the following link:
https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data
1. Load the dataset with following column names ["symboling", "normalized_losses",
"make", "fuel_type", "aspiration","num_doors", "body_style", "drive_wheels",
"engine_location", "wheel_base", "length", "width", "height", "curb_weight",
"engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke",
"compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]
and replace all ? values with NaN
2. Replace all NaN values with central tendency imputation. Drop the rows with NaN
values in price column
3. There are 10 columns in the dataset with non-numeric values. Convert these values to
numeric values using following scheme:
(i) For “num_doors” and “num_cylinders”: convert words (number names) to figures
for e.g., two to 2
(ii) For "body_style", "drive_wheels": use dummy encoding scheme
(iii) For “make”, “aspiration”, “engine_location”,fuel_type: use label encoding
scheme
(iv) For fuel_system: replace values containing string pfi to 1 else all values to 0.
(v) For engine_type: replace values containing string ohc to 1 else all values to 0.
4. Divide the dataset into input features (all columns except price) and output variable
(price). Scale all input features.
5. Train a linear regressor on 70% of data (using inbuilt linear regression function of
Python) and test its performance on remaining 30% of data.
6. Reduce the dimensionality of the feature set using inbuilt PCA decomposition and then
again train a linear regressor on 70% of reduced data (using inbuilt linear regression
function of Python). Does it lead to any performance improvement on test set?

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [18]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"

In [20]:
response = requests.get(url)
response.raise_for_status() # gives error if req fails

soup = BeautifulSoup(response.text, 'html.parser')
data_text = soup.get_text()
rows = [line.split(",") for line in data_text.strip().split("\n")]

columns = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration","num_doors", "body_style",
           "drive_wheels", "engine_location", "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke", "compression_ratio",
           "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]

lastdf = pd.DataFrame(rows, columns=columns)

In [21]:
lastdf.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_cylinders,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.60,168.80,64.10,48.80,2548,dohc,four,130,mpfi,3.47,2.68,9.00,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.60,168.80,64.10,48.80,2548,dohc,four,130,mpfi,3.47,2.68,9.00,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.50,171.20,65.50,52.40,2823,ohcv,six,152,mpfi,2.68,3.47,9.00,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.80,176.60,66.20,54.30,2337,ohc,four,109,mpfi,3.19,3.40,10.00,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.40,176.60,66.40,54.30,2824,ohc,five,136,mpfi,3.19,3.40,8.00,115,5500,18,22,17450


In [22]:
lastdf.replace('?', np.nan, inplace=True)

In [23]:
lastdf.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_cylinders,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.60,168.80,64.10,48.80,2548,dohc,four,130,mpfi,3.47,2.68,9.00,111,5000,21,27,13495
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.60,168.80,64.10,48.80,2548,dohc,four,130,mpfi,3.47,2.68,9.00,111,5000,21,27,16500
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.50,171.20,65.50,52.40,2823,ohcv,six,152,mpfi,2.68,3.47,9.00,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.80,176.60,66.20,54.30,2337,ohc,four,109,mpfi,3.19,3.40,10.00,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.40,176.60,66.40,54.30,2824,ohc,five,136,mpfi,3.19,3.40,8.00,115,5500,18,22,17450


In [33]:
# so we need to replace all NaN values with mode if they are categorical variables
# and with mean if they are numeric values

for col in lastdf.columns:
  if lastdf[col].dtype == "object": # categorical
    lastdf[col] = lastdf[col].fillna(lastdf[col].mode()[0])
  else:
    lastdf[col] = lastdf[col].fillna(lastdf[col].mean())

In [34]:
lastdf = lastdf.dropna(subset=['price'])
lastdf['price'] = lastdf['price'].astype(float)

In [37]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

In [38]:
word_to_num = {'two':2, 'three':3, 'four':4, 'five':5, 'six':6, 'eight':8, 'twelve':12}
lastdf['num_doors'] = lastdf['num_doors'].replace(word_to_num).astype(int)
lastdf['num_cylinders'] = lastdf['num_cylinders'].replace(word_to_num).astype(int)

lastdf = pd.get_dummies(lastdf, columns=['body_style', 'drive_wheels'], drop_first = True)
labels_cols = ['make', 'aspiration', 'engine_location', 'fuel_type']
for col in labels_cols:
  lastdf[col] = LabelEncoder().fit_transform(lastdf[col])

lastdf['fuel_system'] = lastdf['fuel_system'].apply(lambda x: 1 if 'pfi' in x else 0)
lastdf['engine_type'] = lastdf['engine_type'].apply(lambda x: 1 if 'ohc' in x else 0)

X = lastdf.drop('price', axis = 1)
y = lastdf['price']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
r2 = r2_score(y_test,y_pred)
print(f'The r2 score is {r2}')

The r2 score is 0.7895045576733848


In [39]:
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)
X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

lr_pca = LinearRegression()
lr_pca.fit(X_train_pca,y_train)
y_pred_pca = lr_pca.predict(X_test_pca)
r2_pca = r2_score(y_test,y_pred_pca)
print(f'The r2 score after PCA is {r2_pca}')

The r2 score after PCA is 0.7478420860380317
